<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/ninologo.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Sharing data features

## Explore additional data

Once you analyse a corpus, it is likely that you produce data that others can reuse.
Maybe you have defined a set of proper name occurrences, or special numerals, or you have computed part-of-speech assignments.

It is possible to turn these insights into *new features*, i.e. new `.tf` files with values assigned to specific nodes.

## Make your own data

New data is a product of your own methods and computations in the first place.
But how do you turn that data into new TF features?
It turns out that the last step is not that difficult.

If you can shape your data as a mapping (dictionary) from node numbers (integers) to values
(strings or integers), then TF can turn that data into a feature file for you with one command.

## Share your new data
You can then easily share your new features on GitHub, so that your colleagues everywhere
can try it out for themselves.

You can add such data on the fly, by passing a `mod={org}/{repo}/{path}` parameter,
or a bunch of them separated by commas.

If the data is there, it will be auto-downloaded and stored on your machine.

Let's do it.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import collections
import os

from tf.app import use


In [3]:
A = use("oldassyrian:clone", checkout="clone", hoist=globals())
# A = use('oldassyrian', hoist=globals())

# Making data

We illustrate the data creation part by creating a new feature, `ummama`.
The idea is that we mark every sign reading that occurs between `um-ma` and `ma` some where in the first 3 lines of a face.
We want to mark every occurrence of such signs elsewhere in the corpus with `ummama=1`.

We only do it if the sign between the `um-ma` and `ma` (which must be on the same line) is not missing, damaged, or questionable.

The easiest way to get started is to run a query:

In [4]:
query = """
line ln<4
  =: sign reading=um missing# damage# question#
  <: sign reading=ma missing# damage# question#
% the next sign is the one that we are after
  < sign missing# damage# question#
  < sign reading=ma missing# damage# question#
"""

In [5]:
results = A.search(query)

  7.34s 4081 results


In [6]:
A.table(results, end=10)

n,p,line,sign,sign,sign,sign
1,P361248 obverse:2,um-ma _dam-gar3_-ru-tum-ma,um-,ma,_dam-,ma
2,P361248 obverse:2,um-ma _dam-gar3_-ru-tum-ma,um-,ma,gar3_-,ma
3,P361248 obverse:2,um-ma _dam-gar3_-ru-tum-ma,um-,ma,ru-,ma
4,P361248 obverse:2,um-ma _dam-gar3_-ru-tum-ma,um-,ma,tum-,ma
5,P360984 obverse:2,um-ma wa-la2-wa-la2-ma,um-,ma,wa-,ma
6,P360984 obverse:2,um-ma wa-la2-wa-la2-ma,um-,ma,la2-,ma
7,P360984 obverse:2,um-ma wa-la2-wa-la2-ma,um-,ma,wa-,ma
8,P360984 obverse:2,um-ma wa-la2-wa-la2-ma,um-,ma,la2-,ma
9,P361576 obverse:2,um-ma szu-su2-en6-ma,um-,ma,szu-,ma
10,P361576 obverse:2,um-ma szu-su2-en6-ma,um-,ma,su2-,ma


Observe how the signs between `um-ma` and `ma` are picked up, except the damaged `nu` and `ur2`.

First we are collect these readings, and survey the frequencies in the result.

Some signs do not have a reading, but then they have a grapheme.
If they do not have a grapheme, they might be comment signs, and we skip them.

In [7]:
umaReadings = collections.Counter()

# collect

for (line, um, ma1, sign, ma2) in results:
    reading = F.reading.v(sign) or F.grapheme.v(sign)
    if not reading:
        continue
    umaReadings[reading] += 1

# show

print(f"Found {len(umaReadings)} distinct readings")
limit = 20

for (reading, amount) in sorted(
    umaReadings.items(),
    key=lambda x: (-x[1], x[0]),
)[0:limit]:
    print(f"{reading:<6} {amount:>4} x")
print(f" ... and {len(umaReadings) - limit} more ...")

Found 164 distinct readings
a       657 x
szur3   171 x
szu     142 x
i       141 x
szur    137 x
na      132 x
ma      115 x
ku      112 x
di2     101 x
um       96 x
sza      82 x
la2      80 x
ta       78 x
hu       70 x
esz18    69 x
dar      67 x
lum2     63 x
lim      55 x
da       53 x
bu       52 x
 ... and 144 more ...


Now we visit all signs in the whole corpus and check whether their reading or grapheme is in this set.
If so, we give that sign a value 1 in the dictionary `ummama`.

In [8]:
ummama = {}

allSigns = F.otype.s("sign")

for s in allSigns:
    reading = F.reading.v(s) or F.grapheme.v(s)
    if not reading:
        continue
    if reading in umaReadings:
        ummama[s] = 1

print(f"Assigned `ummama=1` to {len(ummama)} sign occurrences out of {len(allSigns)}")

Assigned `ummama=1` to 659791 sign occurrences out of 766501


Note that the majority of all signs also occurs between `um-ma` and `ma` at the start of a document.

Maybe this is an indication that we are not capturing the idea of selecting specific signs,
we may have to strengthen our search criterion.

But that is beyond this tutorial. We suppose these ummama words form a valueable set that we want to share.

# Saving data

The [documentation](https://annotation.github.io/text-fabric/tf/core/fabric.html#tf.core.fabric.FabricCore.save)
explains how to save this data into a text-fabric
data file.

We choose a location where to save it, the `exercises` repository in the `Nino-cunei` organization, in the folder `analysis`.

In order to do this, we restart the TF api, but now with the desired output location in the `locations` parameter.

In [12]:
GITHUB = os.path.expanduser("~/github")
ORG = "Nino-cunei"
REPO = "exercises"
PATH = "ass-analysis"
VERSION = A.version

Note the version: we have built the version against a specific version of the data:

In [13]:
A.version

0.1

Later on, we pass this version on, so that users of our data will get the shared data in exactly the same version as their core data.

We have to specify a bit of metadata for this feature:

In [14]:
metaData = {
    "ummama": dict(
        valueType="int",
        description="reading occurs somewhere between um-ma and ma",
        creator="Dirk Roorda",
    ),
}

Now we can give the save command:

In [15]:
TF.save(
    nodeFeatures=dict(ummama=ummama),
    metaData=metaData,
    location=f"{GITHUB}/{ORG}/{REPO}/{PATH}/tf",
    module=VERSION,
)

  0.00s Exporting 1 node and 0 edge and 0 config features to ~/github/Nino-cunei/exercises/ass-analysis/tf/0.1:
   |     0.68s T ummama               to ~/github/Nino-cunei/exercises/ass-analysis/tf/0.1
  0.68s Exported 1 node features and 0 edge features and 0 config features to ~/github/Nino-cunei/exercises/ass-analysis/tf/0.1


True

# Sharing data

How to share your own data is explained in the
[documentation](https://annotation.github.io/text-fabric/tf/about/datasharing.html).

Here we show it step by step for the `ummama` feature.

If you commit your changes to the exercises repo, and have done a `git push origin master`,
you already have shared your data!

If you want to make a stable release, so that you can keep developing, while your users fall back
on the stable data, you can make a new release.

Go to the GitHub website for that, go to your repo, and click *Releases* and follow the nudges.

If you want to make it even smoother for your users, you can zip the data and attach it as a binary to the release just created.

We need to zip the data in exactly the right directory structure. Text-Fabric can do that for us:

In [16]:
%%sh

text-fabric-zip Nino-cunei/exercises/ass-analysis/tf

True
Create release data for Nino-cunei/exercises/ass-analysis/tf
Found 1 versions
zip files end up in ~/Downloads/Nino-cunei-release/exercises
zipping Nino-cunei/exercises       0.1 with   1 features ==> ass-analysis-tf-0.1.zip


All versions have been zipped, but it works OK if you only attach the newest version to the newest release.

If a user asks for an older version in this release, the system can still find it.

# Use the data

We can use the data by calling it up when we say `use('oldassyrian', ...)`.

Here is how:

In [17]:
A = use(
    "oldassyrian:clone",
    checkout="clone",
    hoist=globals(),
    mod="Nino-cunei/exercises/ass-analysis/tf:clone",
)
# A = use('oldassyrian', hoist=globals(), mod='Nino-cunei/exercises/ass-analysis/tf')

   |     0.88s T ummama               from ~/github/Nino-cunei/exercises/ass-analysis/tf/0.1


Above you see a new section in the feature list: **Nino-cunei/exercises/analysis/tf** with our foreign feature in it: `ummama`.

Now, suppose did not know much about this feature, then we would like to do a few basic checks:

In [18]:
F.ummama.freqList()

((1, 659791),)

We see that the feature has only one value, `1`, and that 182222 nodes have it.

Which nodes have a ummmama feature?

In [19]:
{F.otype.v(n) for n in N.walk() if F.ummama.v(n)}

{'sign'}

Only signs have the feature.

Let's have a look at a table of some ummama signs.

In [20]:
results = A.search(
    """
sign ummama
"""
)

  1.06s 659791 results


In [21]:
A.table(results, start=1, end=20)

n,p,sign
1,P390626 obverse:1,_lugal_
2,P390626 obverse:1,lugal-
3,P390626 obverse:1,ke-
4,P390626 obverse:1,en6
5,P390626 obverse:1,_lugal_
6,P390626 obverse:2,a-
7,P390626 obverse:2,ki-
8,P390626 obverse:2,di2-
9,P390626 obverse:2,e
10,P390626 obverse:2,re-


Now let's get some non-ummama signs:

In [22]:
results = A.search(
    """
sign ummama#
"""
)

  0.59s 106710 results


In [23]:
A.table(results, start=1, end=20)

n,p,sign
1,P390626 obverse:3,le-
2,P390626 obverse:4,wu-
3,P390626 obverse:7,"as,-"
4,P390626 obverse:8,u4-
5,P390626 obverse:8,te2-
6,P390626 obverse:11,iq
7,P390626 obverse:15,iq-
8,P390626 obverse:17,"as,-"
9,P390626 obverse:18,sze2-
10,P390626 obverse:19,gu4


Let's get lines with both ummama and non-ummama signs:

In [24]:
results = A.search(
    """
line
  sign ummama
  sign ummama#
"""
)

  3.13s 607109 results


In [25]:
A.table(results, start=1, end=2, condensed=True)

With highlights:

In [26]:
highlights = {}

for s in F.otype.s("sign"):
    color = "lightsalmon" if F.ummama.v(s) else "mediumaquamarine"
    highlights[s] = color

In [27]:
A.table(
    results, start=1, end=10, baseTypes="sign", condensed=True, highlights=highlights
)

If we do a pretty display, the `ummama` feature shows up.

In [28]:
A.show(
    results,
    start=1,
    end=3,
    baseTypes="sign",
    condensed=True,
    withNodes=True,
    highlights=highlights,
)

Or in the context of a whole face:

In [29]:
A.show(
    results,
    start=1,
    end=1,
    condensed=True,
    condenseType="face",
    withNodes=False,
    highlights=highlights,
)

# All together!

If more researchers have shared data modules, you can draw them all in.

Then you can design queries that use features from all these different sources.

In that way, you build your own research on top of the work of others.

Hover over the features to see where they come from, and you'll see they come from your local github repo.

---

All chapters:

* **[start](start.ipynb)** become an expert in creating pretty displays of your text structures
* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **share** draw in other people's data and let them use yours
* **[similarLines](similarLines.ipynb)** spot the similarities between lines

---

See the [cookbook](cookbook) for recipes for small, concrete tasks.

CC-BY Dirk Roorda